# Modul 3 - Demo

### Dataset : https://drive.google.com/file/d/1x2Xpa5T-ifi1xn-MtKA8HOV7E9Rk0NlJ/view?pli=1

Gunakan dataset di atas untuk melakukan serangkaian proses di bawah:

1. Handling Categorical Values:
- Identifikasi kolom-kolom kategorikal dalam dataset.
- Terapkan metode handling categorical value yang cocok pada 3 kolom yang ada dalam dataset (tidak boleh ketiganya metode yang sama, bisa proporsi 2 dan 1).
- Lakukan Binning pada kolom age dengan membaginya ke dalam 4 kelompok: "Muda", "Dewasa", "Paruh Baya", dan "Lanjut Usia".

2. Data Normalization:
- Gunakan Min-Max Scaling pada kolom balance.
- Gunakan Z-Score Scaling pada kolom duration.
- Gunakan Decimal Scaling pada kolom campaign.

3. Dimensionality Reduction:
- Lakukan Feature Selection dengan memilih hanya fitur yang memiliki korelasi tinggi terhadap variabel target (y). Gunakan korelasi Pearson atau metode seleksi fitur lain yang sesuai.
- Lakukan Feature Extraction menggunakan PCA (Principal Component Analysis) untuk mereduksi dimensi dataset menjadi hanya 5 fitur utama.

4. Data Splitting:
- Bagi dataset menjadi Train (70%), Validation (15%), dan Test (15%).
- Pastikan bahwa distribusi kelas dalam variabel target (y) tetap seimbang dalam proses pembagian data.

### Persiapan Awal

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# Membaca dataset
df = pd.read_csv('tugas3_genap.csv')

# Mengganti 'deposit' menjadi 'y' sebagai variabel target
df = df.rename(columns={'deposit': 'y'})

### 1. Handling Categorical Values

In [2]:
# Bagian 1: Handling Categorical Values

# Identifikasi kolom kategorikal
categorical_cols = df.select_dtypes(include=['object']).columns
print("Kolom kategorikal:", categorical_cols)

# Buat salinan DataFrame
df_categorical = df.copy()

# 1. One-Hot Encoding untuk kolom 'job'
df_categorical = pd.get_dummies(df_categorical, columns=['job'], prefix='job')

# 2. Label Encoding untuk kolom 'marital'
le = LabelEncoder()
df_categorical['marital'] = le.fit_transform(df_categorical['marital'])

# 3. Frequency Encoding untuk kolom 'education'
education_freq = df_categorical['education'].value_counts(normalize=True)
df_categorical['education'] = df_categorical['education'].map(education_freq)

# Binning kolom 'age' menjadi 4 kelompok
bins = [0, 25, 40, 60, np.inf]
labels = ['Muda', 'Dewasa', 'Paruh Baya', 'Lanjut Usia']
df_categorical['age_group'] = pd.cut(df_categorical['age'], bins=bins, labels=labels, include_lowest=True)

# Tampilkan DataFrame hasil
print("DataFrame setelah handling categorical values:")
print(df_categorical.head())

# Simpan hasil ke DataFrame
df_output_1 = df_categorical

Kolom kategorikal: Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome', 'y'],
      dtype='object')
DataFrame setelah handling categorical values:
   age  marital  education default  balance housing loan  contact  day month  \
0   59        1   0.490593      no     2343     yes   no  unknown    5   may   
1   56        1   0.490593      no       45      no   no  unknown    5   may   
2   41        1   0.490593      no     1270     yes   no  unknown    5   may   
3   55        1   0.490593      no     2476     yes   no  unknown    5   may   
4   54        1   0.330496      no      184      no   no  unknown    5   may   

   ...  job_housemaid  job_management  job_retired  job_self-employed  \
0  ...          False           False        False              False   
1  ...          False           False        False              False   
2  ...          False           False        False              False   
3  ...          False     

### 2. Data Normalization

In [3]:
# Bagian 2: Data Normalization

# Gunakan hasil dari bagian 1
df_normalization = df_output_1.copy()

# 1. Min-Max Scaling pada kolom 'balance'
minmax_scaler = MinMaxScaler()
df_normalization['balance_scaled'] = minmax_scaler.fit_transform(df_normalization[['balance']])

# 2. Z-Score Scaling pada kolom 'duration'
zscore_scaler = StandardScaler()
df_normalization['duration_scaled'] = zscore_scaler.fit_transform(df_normalization[['duration']])

# 3. Decimal Scaling pada kolom 'campaign'
j = len(str(df_normalization['campaign'].abs().max()))
df_normalization['campaign_scaled'] = df_normalization['campaign'] / (10 ** j)

# Tampilkan DataFrame hasil
print("DataFrame setelah normalisasi:")
print(df_normalization[['balance', 'balance_scaled', 'duration', 'duration_scaled', 'campaign', 'campaign_scaled']].head())

# Simpan hasil ke DataFrame
df_output_2 = df_normalization

DataFrame setelah normalisasi:
   balance  balance_scaled  duration  duration_scaled  campaign  \
0     2343        0.104371      1042         1.930226         1   
1       45        0.078273      1467         3.154612         1   
2     1270        0.092185      1389         2.929901         1   
3     2476        0.105882       579         0.596366         1   
4      184        0.079851       673         0.867171         2   

   campaign_scaled  
0             0.01  
1             0.01  
2             0.01  
3             0.01  
4             0.02  


### 3. Dimensionality Reduction

In [9]:
# Bagian 3: Dimensionality Reduction

# Gunakan hasil dari bagian 2
df_reduction = df_output_2.copy()

# Ubah variabel target 'y' menjadi numerik
df_reduction['y'] = df_reduction['y'].map({'yes': 1, 'no': 0})

# Pisahkan fitur numerik untuk analisis
numeric_cols = df_reduction.select_dtypes(include=[np.number]).columns
X = df_reduction[numeric_cols].drop(columns=['y'])
y = df_reduction['y']

# 1. Feature Selection berdasarkan korelasi Pearson
correlation = X.corrwith(y).abs()
selected_features = correlation[correlation > 0.1].index  # Ambang batas korelasi 0.1
X_selected = X[selected_features]

print("Fitur yang dipilih berdasarkan korelasi:", selected_features)

# Buat DataFrame dari fitur terpilih
df_selected = pd.DataFrame(X_selected, columns=selected_features)
df_selected['y'] = y.values

# 2. Feature Extraction dengan PCA
X_numeric = X.fillna(0)  # Mengisi NaN dengan 0 jika ada
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_numeric)

# Buat DataFrame dari hasil PCA
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
df_pca['y'] = y.values

# Tampilkan DataFrame hasil PCA
print("DataFrame setelah PCA:")
print(df_pca.head())

# Simpan hasil ke DataFrame
df_output_3 = df_pca

Fitur yang dipilih berdasarkan korelasi: Index(['duration', 'campaign', 'pdays', 'previous', 'duration_scaled',
       'campaign_scaled'],
      dtype='object')
DataFrame setelah PCA:
           PC1          PC2        PC3        PC4        PC5  y
0   816.071910   668.500089 -46.292536  17.517682 -10.663397  1
1 -1480.883660  1099.081240 -40.785722  15.505014 -10.374289  1
2  -256.084399  1018.100653 -42.288583  -0.004865 -10.498372  1
3   947.938769   205.197412 -50.843613  13.427466 -10.921609  1
4 -1343.824594   304.778559 -48.538139  13.385242 -10.753779  1


### 4. Data Splitting

In [5]:
# Bagian 4: Data Splitting

# Gunakan hasil dari bagian 3 (PCA)
df_splitting = df_output_3.copy()

# Pisahkan fitur dan target
X_final = df_splitting.drop(columns=['y'])
y_final = df_splitting['y']

# Split menjadi Train (70%), Validation (15%), Test (15%) dengan stratifikasi
X_temp, X_test, y_temp, y_test = train_test_split(X_final, y_final, test_size=0.15, stratify=y_final, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, stratify=y_temp, random_state=42)  # 0.1765 = 15/(100-15)

# Buat DataFrame untuk masing-masing subset
df_train = pd.DataFrame(X_train, columns=X_final.columns)
df_train['y'] = y_train.values

df_val = pd.DataFrame(X_val, columns=X_final.columns)
df_val['y'] = y_val.values

df_test = pd.DataFrame(X_test, columns=X_final.columns)
df_test['y'] = y_test.values

# Tampilkan DataFrame hasil
print("DataFrame Train:")
print(df_train.head())
print("DataFrame Validation:")
print(df_val.head())
print("DataFrame Test:")
print(df_test.head())

# Verifikasi distribusi
print("Distribusi kelas di Train:", df_train['y'].value_counts(normalize=True))
print("Distribusi kelas di Validation:", df_val['y'].value_counts(normalize=True))
print("Distribusi kelas di Test:", df_test['y'].value_counts(normalize=True))

# Simpan hasil ke DataFrame
df_output_4_train = df_train
df_output_4_val = df_val
df_output_4_test = df_test

DataFrame Train:
              PC1         PC2         PC3        PC4        PC5  y
4451   170.008421 -194.807307   40.748510  -8.288107  -9.535718  1
7201  -230.774607  326.066922  -48.975499   0.934200 -12.809956  0
445  -1337.878673  693.744831  -44.846934   2.389484  -1.499191  1
3022  3784.323357  -99.610985  141.481761  14.171781   1.096521  1
5057  1106.561006   32.085032   22.339948   9.308783   5.486690  1
DataFrame Validation:
              PC1          PC2         PC3        PC4        PC5  y
2944  -609.105417  -240.920692   41.713579  -7.993196   0.407535  1
413  -1522.954776  1071.182037  -41.068284   8.506993  -6.180076  1
6763  -843.241807  -271.427550  -54.549763 -14.953885  13.820093  0
5659 -1164.060457  -199.640073  -53.532893   3.241249  -8.055891  0
3736  -722.590856  -121.155844  405.890934  -6.998934   5.843752  1
DataFrame Test:
               PC1          PC2         PC3       PC4        PC5  y
2169  -1458.832538  -104.921507  -52.400879 -2.559432 -15.052372  1

In [7]:
df_output_3

,PC1,PC2,PC3,PC4,PC5,y
0,816.071910,668.500089,-46.292536,17.517682,-10.663397,1
1,-1480.883660,1099.081240,-40.785722,15.505014,-10.374289,1
2,-256.084399,1018.100653,-42.288583,-0.004865,-10.498372,1
3,947.938769,205.197412,-50.843613,13.427466,-10.921609,1
4,-1343.824594,304.778559,-48.538139,13.385242,-10.753779,1
...,...,...,...,...,...,...
11157,-1527.848526,-110.761190,-52.532257,-7.629737,3.910674,0
11158,-796.273616,-286.536822,-54.642436,-1.917577,-0.043215,0
11159,-1500.095802,-211.824609,-53.520939,-8.647919,2.901507,0
11160,-1529.348799,-360.414800,118.177029,2.397092,-7.077395,0
